In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

data_fake = pd.read_csv("C:/Users/hp/Downloads/Fake.csv/Fake.csv")
data_true = pd.read_csv("C:/Users/hp/Downloads/True.csv/True.csv")


In [4]:
data_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [5]:
data_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [9]:
# Assign labels
data_fake['class'] = 0
data_true['class'] = 1

# ✅ Make copies of last 10 rows for manual testing
data_fake_manual_testing = data_fake.tail(10).copy()
data_true_manual_testing = data_true.tail(10).copy()

# ✅ Drop last 10 rows using iloc to avoid index errors
data_fake = data_fake.iloc[:-10].copy()
data_true = data_true.iloc[:-10].copy()

# ✅ No warning now when modifying manual testing data
data_fake_manual_testing['class'] = 0
data_true_manual_testing['class'] = 1

# Check new shapes
print(data_fake.shape, data_true.shape)


(23461, 5) (21397, 5)


In [10]:
data_fake.shape, data_true.shape
data_fake_manual_testing

,title,text,subject,date,class
23461,REPORT: ‘Federal Government Escalated the Viol...,KILLED: Rancher and protest spokesman Robert ...,Middle-east,"January 28, 2016",0
23462,"BOILER ROOM – Oregon Standoff, Cuddle Parties,...",Tune in to the Alternate Current Radio Network...,Middle-east,"January 28, 2016",0
23463,"Eyewitness Says Feds Ambushed Bundys, 100 Shot...",Patrick Henningsen 21st Century Wire UPDATE: 1...,Middle-east,"January 27, 2016",0
23464,Episode #119 – SUNDAY WIRE: ‘You Know the Dril...,Episode #119 of SUNDAY WIRE SHOW finally resum...,Middle-east,"January 24, 2016",0
23465,‘There’ll be boots on the ground’: US making n...,21st Century Wire says Various parties in Wash...,Middle-east,"January 23, 2016",0
23466,Boston Brakes? How to Hack a New Car With Your...,21st Century Wire says For those who still ref...,Middle-east,"January 22, 2016",0
23467,Oregon Governor Says Feds ‘Must Act’ Against P...,"21st Century Wire says So far, after nearly 20...",Middle-east,"January 21, 2016",0
23468,Ron Paul on Burns Oregon Standoff and Jury Nul...,21st Century Wire says If you ve been followin...,Middle-east,"January 21, 2016",0
23469,BOILER ROOM: As the Frogs Slowly Boil – EP #40,Tune in to the Alternate Current Radio Network...,Middle-east,"January 20, 2016",0
23470,Arizona Rancher Protesting in Oregon is Target...,RTOne of the most visible members of the armed...,Middle-east,"January 20, 2016",0


In [12]:
import pandas as pd
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# Preprocessing function
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', ' ', text)
    text = re.sub('\W', ' ', text)
    text = re.sub('https?://\S+|www\.\S+', ' ', text)
    text = re.sub('<.*?>+', ' ', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\n', ' ', text)
    text = re.sub('\w*\d\w*', ' ', text)
    return text

# Combine fake and true datasets
data = pd.concat([data_fake, data_true], axis=0)
data = data.sample(frac=1).reset_index(drop=True)

# Apply preprocessing
data['text'] = data['text'].apply(wordopt)

# Feature and label separation
x = data['text']
y = data['class']

# Train-test split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

# TF-IDF Vectorization
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

# Logistic Regression Model
LR = LogisticRegression()
LR.fit(xv_train, y_train)

# Prediction
pred_lr = LR.predict(xv_test)

# Accuracy
accuracy = LR.score(xv_test, y_test)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9860900579580918


In [ ]:
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier

# Logistic Regression classification report
print("Logistic Regression Report:")
print(classification_report(y_test, pred_lr))

# Decision Tree
DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)
pred_dt = DT.predict(xv_test)
print("\nDecision Tree Report:")
print(classification_report(y_test, pred_dt))

# Gradient Boosting
GB = GradientBoostingClassifier(random_state=0)
GB.fit(xv_train, y_train)
pred_gb = GB.predict(xv_test)
print("\nGradient Boosting Report:")
print(classification_report(y_test, pred_gb))

# Random Forest
RF = RandomForestClassifier(random_state=0)
RF.fit(xv_train, y_train)
pred_rf = RF.predict(xv_test)
print("\nRandom Forest Report:")
print(classification_report(y_test, pred_rf))

# Label output function
def output_label(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not A Fake News"

# Manual test function
def manual_testing(news):
    testing_news = {"text": [news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt)
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)

    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_GB = GB.predict(new_xv_test)
    pred_RF = RF.predict(new_xv_test)

    print("\nManual Testing Results:")
    print("LR Prediction:", output_label(pred_LR[0]))
    print("DT Prediction:", output_label(pred_DT[0]))
    print("GB Prediction:", output_label(pred_GB[0]))
    print("RF Prediction:", output_label(pred_RF[0]))


Logistic Regression Report:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5941
           1       0.98      0.99      0.99      5274

    accuracy                           0.99     11215
   macro avg       0.99      0.99      0.99     11215
weighted avg       0.99      0.99      0.99     11215


Decision Tree Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5941
           1       1.00      0.99      1.00      5274

    accuracy                           1.00     11215
   macro avg       1.00      1.00      1.00     11215
weighted avg       1.00      1.00      1.00     11215

